In [2]:
import os
import sys

import torch
import torch.nn as nn
import numpy as np
import torch.nn.functional as F  
import matplotlib 
import matplotlib.pyplot as plt

In [3]:
sys.path.append("/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/models") #probably keep this still
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/justin_model_slicing_norm_LR1_smallHop_12_largerInput.pth" #probably keep this still
#IQ_FILE_PATH = "/data2/brandan/RFSPOOF/brandan/1m_targeted/run0.iq" 

MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/best_model.pth"

In [3]:
'''
file_list = [
    "brandan_2/pluto0_1m_adv_2m_run0.iq",
    "brandan_2/pluto0_1m_adv_2m_run1.iq",
    "brandan_2/pluto0_1mLeft_adv_1mRight_run0.iq",
    "brandan_2/pluto0_1mLeft_adv_1mRight_run1.iq",
    "brandan_2/pluto0_2m_adv_1m_run0.iq",
    "brandan_2/pluto0_2m_adv_1m_run1.iq",
    "brandan_2/pluto0_2m_adv_2m_run0.iq",
    "brandan_2/pluto0_2m_adv_2m_run1.iq",
    "brandan_2/pluto0_2mLeft_adv_2mRight_run0.iq",
    "brandan_2/pluto0_2mLeft_adv_2mRight_run1.iq"
]
'''

file_list = [
    "brandan_4_1/2m_adv2m_eot.iq",
    "brandan_4_1/2m_adv2m.iq",
    "brandan_4_1/2m_adv3m_eot.iq",
    "brandan_4_1/2m_adv3m.iq",
    "brandan_4_2/2m_adv4m_eot.iq",
    "brandan_4_2/2m_adv4m.iq",
    "brandan_4_2/2m_advside2.5m_eot.iq",
    "brandan_4_2/2m_advside2.5m.iq"
    
    
]

In [ ]:
from attempt2 import resnet50_1d  
num_classes = 8 

model = resnet50_1d(num_classes=num_classes).to(DEVICE)

MODEL_PATH = "/home/jfeng/Desktop/jfeng/rf_spoofing/spoofing/weights/justin_model_slicing_norm_LR1_smallHop_12.pth" #probably keep this still
#print(f"Loading trained model weights from: {MODEL_PATH}")
state_dict = torch.load(MODEL_PATH, map_location=DEVICE)

model.load_state_dict(state_dict)

model.eval()

In [5]:
def load_iq_data(file_path, start_idx=0, max_samples=4096*2):
    total_samples = max_samples * 2 
    with open(file_path, "rb") as f:
        f.seek(start_idx * 4 * 2)  
        raw_data = np.fromfile(f, dtype="float32", count=total_samples)

    if raw_data.shape[0] < total_samples:
        raise ValueError(f"Not enough data in {file_path}. Requested {total_samples}, got {raw_data.shape[0]}.")

    #  Extract I/Q channels
    I = raw_data[0::2]  
    Q = raw_data[1::2]  

    iq_data = np.stack([I, Q], axis=0)

    iq_data = np.expand_dims(iq_data, axis=0)

    data_tensor = torch.from_numpy(iq_data).float()

    label_tensor = torch.tensor([0], dtype=torch.long)

    return data_tensor, label_tensor #loading IQ samples i got from chatgpt

In [7]:
'''
this is code that tests each batch of 8912 samples and aggregates the accuracy
'''
individual_path = "brandan/1m_targeted/run0.iq"
file_size = os.path.getsize(individual_path) 
total_samples = file_size // (4 * 2)  
max_samples = 4096 * 2  
total_chunks = total_samples // max_samples 

print(f"Total samples: {total_samples}, Total chunks: {total_chunks}")


start_idx = max(0, total_chunks - 10)  


count3 = 0
avg_conf = 0
for idx in range(0, total_chunks):
    data, labels = load_iq_data(individual_path, start_idx=idx * max_samples)
    data, labels = data.to(DEVICE), labels.to(DEVICE)
    
    with torch.no_grad():
        raw_output = model(data)
        probabilities = F.softmax(raw_output, dim=1) #softmax prob to 0-1 prob
        prediction = torch.argmax(probabilities, dim=1)  #take max prediction and confidence
        confidence = probabilities.max(dim=1).values
        
        if prediction == 3:
            avg_conf += confidence
            count3+=1
            
    
    #print(f"prediction: {prediction.cpu().numpy()}, confidence = {confidence.cpu().item():.4f}, ")

print(f"avg confidence = {avg_conf/count3}, amt of chunks classified as 3 is {count3}")


Total samples: 42697200, Total chunks: 5212
avg confidence = tensor([0.9559], device='cuda:0'), amt of 3 is 4832


In [6]:
for path in file_list:
    file_size = os.path.getsize(path) 
    total_samples = file_size // (4 * 2)  
    max_samples = 4096 * 2  
    total_chunks = total_samples // max_samples 

    print(f"Total samples: {total_samples}, Total chunks: {total_chunks}")

    count3 = 0
    conf_values = []  # Store confidence values for class 3

    for idx in range(0, total_chunks):
        data, labels = load_iq_data(path, start_idx=idx * max_samples)
        data, labels = data.to(DEVICE), labels.to(DEVICE)

        with torch.no_grad():
            raw_output = model(data)
            probabilities = F.softmax(raw_output, dim=1)  # Convert logits to probabilities
            prediction = torch.argmax(probabilities, dim=1)  # Get predicted class
            confidence = probabilities.max(dim=1).values  # Get confidence score

            mask = prediction == 3  # Mask for class 3
            if mask.any():
                count3 += mask.sum().item()
                conf_values.extend(confidence[mask].cpu().tolist())  # Convert to list

    if count3 > 0:
        avg_conf = sum(conf_values) / count3  # Compute mean
        std_dev = torch.std(torch.tensor(conf_values), unbiased=True).item() if len(conf_values) > 1 else 0  # Compute std deviation
    else:
        avg_conf = 0
        std_dev = 0

    print(f"File path: {path}, avg confidence = {avg_conf:.4f}, std deviation = {std_dev:.4f}, amt of 3 is {count3}")

Total samples: 41928120, Total chunks: 5118
File path: brandan_4_1/2m_adv2m_eot.iq, avg confidence = 0.9842, std deviation = 0.0503, amt of 3 is 5031
Total samples: 41669040, Total chunks: 5086
File path: brandan_4_1/2m_adv2m.iq, avg confidence = 0.9871, std deviation = 0.0484, amt of 3 is 4958
Total samples: 41746560, Total chunks: 5096
File path: brandan_4_1/2m_adv3m_eot.iq, avg confidence = 0.9817, std deviation = 0.0532, amt of 3 is 5006
Total samples: 41654760, Total chunks: 5084
File path: brandan_4_1/2m_adv3m.iq, avg confidence = 0.9684, std deviation = 0.0781, amt of 3 is 4807
Total samples: 41491560, Total chunks: 5064
File path: brandan_4_2/2m_adv4m_eot.iq, avg confidence = 0.9354, std deviation = 0.1043, amt of 3 is 4572
Total samples: 41503800, Total chunks: 5066
File path: brandan_4_2/2m_adv4m.iq, avg confidence = 0.9516, std deviation = 0.0896, amt of 3 is 4694
Total samples: 41750640, Total chunks: 5096
File path: brandan_4_2/2m_advside2.5m_eot.iq, avg confidence = 0.894

In [7]:
'''
this is code that tests all the first 8912 samples of data
for IQ_FILE_PATH in file_list:
    # Load data and move to device
    data, labels = load_iq_data(IQ_FILE_PATH)
    data, labels = data.to(DEVICE), labels.to(DEVICE)
    
    print(f"The File: {IQ_FILE_PATH}")
    #print("Original data shape:", data.shape)
    
    # Run the model on the data
    with torch.no_grad():
        logits = model(data)                # Raw model output
        probs = F.softmax(logits, dim=1)      # Convert to probabilities
        original_pred = torch.argmax(probs, dim=1)  # Predicted class
        confidence = probs.max(dim=1).values       # Confidence score
    
    # Print the results
    print(f"Prediction: {original_pred.cpu().numpy()}, Confidence: {confidence.cpu().item():.4f}\n")
    '''

'\nthis is code that tests all the first 8912 samples of data\nfor IQ_FILE_PATH in file_list:\n    # Load data and move to device\n    data, labels = load_iq_data(IQ_FILE_PATH)\n    data, labels = data.to(DEVICE), labels.to(DEVICE)\n    \n    print(f"The File: {IQ_FILE_PATH}")\n    #print("Original data shape:", data.shape)\n    \n    # Run the model on the data\n    with torch.no_grad():\n        logits = model(data)                # Raw model output\n        probs = F.softmax(logits, dim=1)      # Convert to probabilities\n        original_pred = torch.argmax(probs, dim=1)  # Predicted class\n        confidence = probs.max(dim=1).values       # Confidence score\n    \n    # Print the results\n    print(f"Prediction: {original_pred.cpu().numpy()}, Confidence: {confidence.cpu().item():.4f}\n")\n    '

In [8]:
'''Adv_path = "/data2/brandan/RFSPOOF/brandan/1m_targeted/run0.iq" 
Channel_path = "/data2/brandan/RFSPOOF/brandan/adversary_noise/run0.iq" 

Adv_data, Adv_labels = load_iq_data(Adv_path)
Adv_data, Adv_labels = data.to(DEVICE), labels.to(DEVICE)

Channel_data, Channel_labels = load_iq_data(Channel_path)
Channel_data, Channel_labels = data.to(DEVICE), labels.to(DEVICE)\

Adv_data_np = Adv_data.cpu().detach().numpy()
Channel_data_np = Channel_data.cpu().detach().numpy()

I_Channel = Adv_data_np[0, 0, :]  
Q_Channel = Adv_data_np[0, 1, :]  

I_Adv = Channel_data_np[0, 0, :] 
Q_Adv = Channel_data_np[0, 1, :] 

time = np.arange(I_Channel.shape[0])

downsample_factor = 100  # Adjust as needed
time = time[::downsample_factor]'''

'Adv_path = "/data2/brandan/RFSPOOF/brandan/1m_targeted/run0.iq" \nChannel_path = "/data2/brandan/RFSPOOF/brandan/adversary_noise/run0.iq" \n\nAdv_data, Adv_labels = load_iq_data(Adv_path)\nAdv_data, Adv_labels = data.to(DEVICE), labels.to(DEVICE)\n\nChannel_data, Channel_labels = load_iq_data(Channel_path)\nChannel_data, Channel_labels = data.to(DEVICE), labels.to(DEVICE)\nAdv_data_np = Adv_data.cpu().detach().numpy()\nChannel_data_np = Channel_data.cpu().detach().numpy()\n\nI_Channel = Adv_data_np[0, 0, :]  \nQ_Channel = Adv_data_np[0, 1, :]  \n\nI_Adv = Channel_data_np[0, 0, :] \nQ_Adv = Channel_data_np[0, 1, :] \n\ntime = np.arange(I_Channel.shape[0])\n\ndownsample_factor = 100  # Adjust as needed\ntime = time[::downsample_factor]'

In [9]:
'''plt.subplot(3, 1, 1)
plt.plot(time, I_Adv, label="Original I", alpha=0.7)
plt.plot(time, I_Channel, label="Adversarial I", linestyle="dashed", alpha=0.7)
plt.title("Time-Domain Signal: Original vs. Adversarial (I Channel)")
plt.xlabel("Sample Index")
plt.ylabel("Amplitude")
plt.legend()'''

'plt.subplot(3, 1, 1)\nplt.plot(time, I_Adv, label="Original I", alpha=0.7)\nplt.plot(time, I_Channel, label="Adversarial I", linestyle="dashed", alpha=0.7)\nplt.title("Time-Domain Signal: Original vs. Adversarial (I Channel)")\nplt.xlabel("Sample Index")\nplt.ylabel("Amplitude")\nplt.legend()'

In [ ]:
'''
this is code that loads data the way justin had it before

def load_iq_data(file_path):
    """
    Loads an IQ binary file and extracts real/imaginary components.

    Args:
        file_path (str): Path to the `.iq` file.

    Returns:
        np.array: Data in shape (2, sequence_length).
    """
    with open(file_path, "rb") as f:
        raw_data = np.fromfile(f, dtype="float32")

    size_data = len(raw_data)
    size_half = size_data // 2
    real_part = raw_data[:size_half]  # I (In-phase)
    imag_part = raw_data[size_half:]  # Q (Quadrature)

    return real_part, imag_part

# **Dataset Class (Using Training Processing)**
class IQDataset(Dataset):
    def __init__(self, file_list):
        self.file_list = file_list
        self.data = []
        self.labels = []
        self.load_files()

    def load_files(self):
        """
        Loads all IQ files and slices them into overlapping test segments.
        """
        for i, file_path in enumerate(self.file_list):
            print(f"Processing: {file_path}")
            
            # Load IQ data
            real_part, imag_part = load_iq_data(file_path)

            # **Apply the same slicing logic as during training**
            for x in range(4800, 6000):  # Matching test range in training
                combined_data = np.vstack((
                    real_part[(x+1)*500:(x+1)*500+10000], 
                    imag_part[(x+1)*500:(x+1)*500+10000]
                ))
                self.data.append(combined_data)
                self.labels.append(i)  # Assign label based on file index

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        sample = self.data[idx]
        label = self.labels[idx]

        sample = torch.from_numpy(sample).float()
        # Normalize (Same as Training)
        magnitude = torch.sqrt(torch.sum(sample**2, dim=1, keepdim=True))
        sample = sample / (magnitude + 1e-10)  # Avoid division by zero

        label_tensor = torch.tensor(label, dtype=torch.long)

        return sample, label_tensor

# **Load dataset & DataLoader**
test_dataset = IQDataset(file_list)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

# **Run Inference on Test Data**
correct = 0
total = 0
all_confidences = []
all_predictions = []

with torch.no_grad():
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.to(DEVICE), labels.to(DEVICE)

        # Run model inference
        outputs = model(inputs)
        probs = F.softmax(outputs, dim=1)  # Convert logits to probabilities
        predicted = torch.argmax(probs, dim=1)  # Predicted class
        confidence = probs.max(dim=1).values  # Confidence score

        total += labels.size(0)
        correct += (predicted == labels).sum().item()

        # Store predictions & confidence scores
        all_predictions.extend(predicted.cpu().numpy())
        all_confidences.extend(confidence.cpu().numpy())

# Compute final accuracy
accuracy = correct / total * 100
print(f'Final Test Accuracy: {accuracy:.2f}%')

# **Print sample predictions & confidence scores**
for i, file in enumerate(file_list):
    print(f"File: {file}")
    print(f"Predicted Class: {all_predictions[i]}, Confidence: {all_confidences[i]:.4f}\n")
'''

Processing: brandan_2/pluto0_1m_adv_2m_run0.iq
Processing: brandan_2/pluto0_1m_adv_2m_run1.iq
Processing: brandan_2/pluto0_1mLeft_adv_1mRight_run0.iq
Processing: brandan_2/pluto0_1mLeft_adv_1mRight_run1.iq
Processing: brandan_2/pluto0_2m_adv_1m_run0.iq
Processing: brandan_2/pluto0_2m_adv_1m_run1.iq
Processing: brandan_2/pluto0_2m_adv_2m_run0.iq
Processing: brandan_2/pluto0_2m_adv_2m_run1.iq
Processing: brandan_2/pluto0_2mLeft_adv_2mRight_run0.iq
Processing: brandan_2/pluto0_2mLeft_adv_2mRight_run1.iq
Final Test Accuracy: 16.57%
File: brandan_2/pluto0_1m_adv_2m_run0.iq
Predicted Class: 4, Confidence: 0.3557

File: brandan_2/pluto0_1m_adv_2m_run1.iq
Predicted Class: 4, Confidence: 0.5250

File: brandan_2/pluto0_1mLeft_adv_1mRight_run0.iq
Predicted Class: 4, Confidence: 0.3290

File: brandan_2/pluto0_1mLeft_adv_1mRight_run1.iq
Predicted Class: 4, Confidence: 0.3636

File: brandan_2/pluto0_2m_adv_1m_run0.iq
Predicted Class: 4, Confidence: 0.3741

File: brandan_2/pluto0_2m_adv_1m_run1.iq
Pr

In [ ]:
File Path,Total Samples,Total Chunks,Avg Confidence,Std Deviation,Amount of 3

